# File to process data from uni forms survey
## from importing data to processing and visualizing

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importing data

In [ ]:
# Importing data
df_uni_data = pd.read_csv('data/uni_forms.csv')

## Data Exploration

In [ ]:
# Displaying the first 20 rows of the data
df_uni_data.head(20)

## Data Cleaning

In [ ]:
# Checking for missing values
df_uni_data.isnull().sum()

In [ ]:
# Dropping the missing values
df_uni_data.dropna(inplace=True)

In [ ]:
# Checking for duplicates
df_uni_data.duplicated().sum()

In [ ]:
# Dropping duplicates
df_uni_data.drop_duplicates(inplace=True)

In [ ]:
# Checking for unique values in the data
df_uni_data.nunique()

In [ ]:
# Replacing the missing values with the average values and fitting strings
df_uni_data.fillna(df_uni_data.mean(), inplace=True)

## Data Preprocessing

In [ ]:
# Checking for the data types
df_uni_data.dtypes

## Data Analysis

In [ ]:
# Checking the distribution of the data
df_uni_data.describe()

## Data Visualization

In [ ]:
# Plotting the distribution of the data
df_uni_data.hist(figsize=(20, 20))
plt.show()
# add seaborn style
sns.set()
# add title and labels
plt.title('Distribution of the data')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()